In [0]:
!pip install --quiet dbdemos

In [0]:
%pip install --quiet databricks-dlt

In [0]:
# Note: you may need to restart the kernel using %restart_python or 

dbutils.library.restartPython()
import dbdemos
dbdemos.install('lakehouse-iot-platform', catalog='main', schema='dbdemos_iot_turbine')

In [0]:
display(spark.read.json('/Volumes/main/dbdemos_iot_turbine/turbine_raw_landing/turbine'))

In [0]:
display(spark.read.json('/Volumes/main/dbdemos_iot_turbine/turbine_raw_landing/historical_turbine_status')) #Historical turbine status analyzed

In [0]:
# %sql

# CREATE STREAMING TABLE turbine (
#   CONSTRAINT correct_schema EXPECT (_rescued_data IS NULL)
# )
# COMMENT "Turbine details, with location, wind turbine model type etc"
# AS SELECT * FROM cloud_files("/Volumes/main/dbdemos_iot_turbine/turbine_raw_landing/turbine", "json", map("cloudFiles.inferColumnTypes" , "true"))

In [0]:
import dlt
dlt.enable_local_execution()


In [0]:
@dlt.table(
    name="turbine",
    comment="Turbine details, with location, wind turbine model type etc"
)
def turbine():
    return (
        spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("cloudFiles.inferColumnTypes", "true")
        .load("/Volumes/main/dbdemos_iot_turbine/turbine_raw_landing/turbine")
    )

In [0]:
# %sql
# CREATE STREAMING TABLE sensor_bronze (
#   CONSTRAINT correct_schema EXPECT (_rescued_data IS NULL),
#   CONSTRAINT correct_energy EXPECT (energy IS NOT NULL and energy > 0) ON VIOLATION DROP ROW
# )
# COMMENT "Raw sensor data coming from json files ingested in incremental with Auto Loader: vibration, energy produced etc. 1 point every X sec per sensor."
# AS SELECT * FROM cloud_files("/Volumes/main/dbdemos_iot_turbine/turbine_raw_landing/incoming_data", "parquet", map("cloudFiles.inferColumnTypes" , "true"))

@dlt.table(
    name="sensor_bronze",
    comment="Raw sensor data coming from json files ingested in incremental with Auto Loader: vibration, energy produced etc. 1 point every X sec per sensor."
)
def sensor_bronze():
    return (
        spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("cloudFiles.inferColumnTypes", "true")
        .load("/Volumes/main/dbdemos_iot_turbine/turbine_raw_landing/incoming_data")
    )